# Project Notebook
Generated from script.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to sys.path
# Assuming notebook is in 'notebooks/' and project root is one level up
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
from pokemon_predictor import config
from pokemon_predictor.data_pipeline.tabular import load_data
import pandas as pd
import numpy as np
import joblib


# Advanced XGBoost Tuning

In [ ]:
X_train, X_test, y_train, y_test, classes = load_data('rgb', split_data=True)
    
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import make_scorer, f1_score

param_dist = {
    'estimator__n_estimators': [100, 300, 500, 1000],
    'estimator__max_depth': [3, 4, 5, 6, 8],
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.3],
    'estimator__subsample': [0.6, 0.8, 1.0],
    'estimator__colsample_bytree': [0.6, 0.8, 1.0]
}

clf = MultiOutputClassifier(XGBClassifier(eval_metric='logloss', use_label_encoder=False))
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=20, cv=3, scoring=make_scorer(f1_score, average='macro'), verbose=1, n_jobs=-1)

random_search.fit(X_train, y_train)
print(f"Best Params: {random_search.best_params_}")
print(f"Best Score: {random_search.best_score_}")
